# Optimization of reactor output
This script uses real weather data between 2013 and 2020 to simulate plant output as a function of wind turbines, solar panels, battery size, and three control parameters that change the battery set point based on forecasted energy production. Multiple simulations structured as a DOE are run for different scenarios, and a model is generated to predict the profit as a function of the aforementioned factors. Finally, this model is optimized to find the most profitable configuration for the plant.

The code below is also contained in long_term_output.py. They are duplicated in this Jupyter notebook to take the user step-by-step through the analysis.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import doe_functions as doef
import warnings

warnings.filterwarnings("ignore")

The first pass at the DOE is below. Note that all features are 3 levels except for the wind turbine (wt). This is because some  research suggests 1MW is one of the smallest standard sizes for wind turbines, and that this is quite expensive and powerful relative to our purposes. We effectively structure the DOE as if this is a binary variable and we do a full factorial plus a Box-Behnken design at both levels of the wind turbine feature. When analyzing later, however, this variable will be categorized as continuous to allow the model to signal to us if a smaller wind turbine is optimal (quick research suggests that smaller wind turbines do exist, although their availability is unclear).

In [2]:
parameters = {
    "wt_list" : [0, 1], # number of 1MW wind turbines
    "sp_list" : [5000, 10000, 15000], # area in m2 of solar panels
    "b_list" : [516, 1144, 2288], # battery sizes in kW
    "c1_list" : [0, 1, 2], # constants for battery setpoint eqn
    "c2_list" : [0, 1, 2],
    "c3_list" : [-1, 0, 1]
    }

# DOE input is a 2-level full factorial plus a Box-Behnken to capture curvature
doe = pd.read_excel("DOE.xlsx")

# Run DOE
doe_results, forecast_store = doef.run_doe(doe, parameters, show_run_status = False) 

In [3]:
doe_results.head()

,wt_level,sp_level,b_level,c1_level,c2_level,c3_level,profit,revenue,opex,capex,total_sx,e_to_grid,e_from_grid
0,0,0,0,0,0,0,59098.402663,344256.415447,142648.921875,142509.090909,286880.346206,0.000000e+00,4.564766e+06
1,0,0,0,0,0,2,46757.833526,323337.803426,134070.878991,142509.090909,269448.169522,0.000000e+00,4.290268e+06
2,0,0,0,0,2,0,43315.597330,318047.905667,132223.217428,142509.090909,265039.921389,0.000000e+00,4.231143e+06
3,0,0,0,0,2,2,36176.131643,303454.130041,124768.907488,142509.090909,252878.441701,3.981881e-11,3.992605e+06
4,0,0,0,2,0,0,49798.300094,326294.839646,133987.448643,142509.090909,271912.366371,0.000000e+00,4.287598e+06


## Fitting a model to the DOE
We use polynomial regression to fit a model of degree 2 with interactions to the DOE results. In the function below, we also include the ability to eliminate all higher-order terms of a specific feature. This will come in useful later.

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
import re

# Note that some variables have the suffix _f (for fixed) or _uf (for unfixed).
# This indicates whether the list/array shrinks as variables are eliminated.

def fit_results(doe_results, remove_var=None):
    X_features = doe_results[["wt_level", "sp_level", "b_level", "c1_level", "c2_level", "c3_level"]]
    y = doe_results["profit"]
    
    model = PolynomialFeatures(degree=2)
    
    fit_tr_model = model.fit_transform(X_features)
    features = model.get_feature_names_out()
    X = pd.DataFrame(fit_tr_model, columns=features)
    
    # Remove higher order terms of 'remove_var'. Leave the first order term.
    if remove_var:
        indicies_to_remove = []
        for i in range(len(features)):
            feature = features[i] 
            
            # Searching for " " or "^" ensures only higher-order terms are removed
            if remove_var in feature and (" " in feature or "^" in feature):
                indicies_to_remove.append(i) 
                X.drop(columns=[feature], inplace=True)
                    
        features = np.delete(features, indicies_to_remove)
    
    lr_model = LinearRegression()
    lr_model.fit(fit_tr_model, y)
    
    est = sm.OLS(y, X)
    est_fit = est.fit()

    return X, y, est_fit, features

X, y, est_fit, feature_names = fit_results(doe_results)

print("Initial results:")
print(est_fit.summary())

Initial results:
                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     2076.
Date:                Thu, 10 Aug 2023   Prob (F-statistic):          1.02e-145
Time:                        12:03:52   Log-Likelihood:                -1742.1
No. Observations:                 146   AIC:                             3538.
Df Residuals:                     119   BIC:                             3619.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
1                  1.

## Feature elimination
In the initial results we see that there are there are some features with a low p-value (< 0.05), and others with high p-values (> 0.05). In order to create a better-fitting model, we will eliminate the insignificant features one by one in a process called *backward elimination*.

This is an iterative process in which the feature with the highest eligible p-value is removed and the model is re-run until all features have a significant p-value or are uneligible for removal. A feature is uneligible for removal if:
* It is a first-order feature and there are higher-order features still in the model that contain the first-order feature.
* It is an interaction term and there are higher-order features still in the model that contain one of the features present in the interaction.

In order to check whether a feature is eligible when doing backward elimination, we first make a boolean matrix with each feature in the model, where a row reads True if the corresponding feature is a lower-order feature of the feature in the column. A feature will only be eligible if the matrix reads False for all features still in the model.

Note that higher-order features are identified by searching for the carrot "^" symbol, and reactions by looking for the space " " as this is the convention that sklearn.preprocessing.PolynomialFeatures uses when naming the features.

In [5]:
def get_index(a, b):
    for i in range(len(a)):
       if a[i] == b:
           return i

# Create boolean array that indicates thes lower order terms for each term in the model
def make_lot_array(feature_names):
    lower_order_terms = np.full((len(feature_names), len(feature_names)), False)

    for i in range(len(feature_names)):
        feature = feature_names[i]
        
        # find 1st order terms for squared features
        if "^" in feature:
            first_order_term = re.search("^[a-z0-9_]*", feature).group(0)  
            first_order_term_index = get_index(feature_names, first_order_term)
            lower_order_terms[i, first_order_term_index] = True
                
        # find 1st order terms for interactions
        if " " in feature:
            first_order_term1 = re.search("^[a-z0-9_]*", feature).group(0)
            first_order_term2 = re.search("[a-z0-9_]*$", feature).group(0)      
            first_order_term1_index = get_index(feature_names, first_order_term1)
            first_order_term2_index = get_index(feature_names, first_order_term2)
            lower_order_terms[i, first_order_term1_index] = True
            lower_order_terms[i, first_order_term2_index] = True
            
            # find interactions that contain a 1st order term also present in squared term
            feature1_squared = first_order_term1 + "^2"
            feature2_squared = first_order_term2 + "^2"
            feature1_squared_index = get_index(feature_names, feature1_squared)
            feature2_squared_index = get_index(feature_names, feature2_squared)
            lower_order_terms[feature1_squared_index, i] = True
            lower_order_terms[feature2_squared_index, i] = True
            
    return lower_order_terms

In [6]:
# Deletes highest p-value terms one at a time, if no higher-order terms exist,
# until all (elegible) terms have a p-value of < 0.05
def backward_elimination(est_fit, X, y, feature_names):
    pvals = est_fit.pvalues
    
    lower_order_terms_f = make_lot_array(feature_names)
    inds_of_remaining_terms = list(range(len(feature_names)))

    # Matches index to feature in lower_order_terms_f
    feature_index_dict_f = {feature_names[i]:i for i in range(len(feature_names))}

    feature_names_uf = feature_names
    
    while max(pvals[1:]) > 0.05:

        highest_pval_index = np.argmax(pvals[1:]) + 1
        feature_to_remove = feature_names_uf[highest_pval_index]
        feature_index = feature_index_dict_f[feature_to_remove]
        
        if not any(lower_order_terms_f[inds_of_remaining_terms, feature_index]):
            X.drop(columns=[feature_to_remove], inplace=True)
            feature_names_uf = np.delete(feature_names_uf, highest_pval_index)
            inds_of_remaining_terms[feature_index] = 0
            
            est_fit = sm.OLS(y, X).fit()
            pvals = est_fit.pvalues
        else:
            pvals[highest_pval_index] = 0
        
    return est_fit, feature_names_uf

## Generate a model for profit as a function of the input parameters
Finally, we generate the model by running fit_results and backward_elimination in series, and then extracting the coeficients from the model objects.

In [7]:
# Generate a pd.Series of coefficients for only the significant factors and 
# their 1st-order terms
def generate_sig_model(doe_results, remove_var=None):
    
    X, y, est_fit, feature_names = fit_results(doe_results, remove_var=remove_var)  
    est_sig_fit, feature_names = backward_elimination(est_fit, X, y, feature_names)
    
    coefs = est_sig_fit.params
    
    print(est_sig_fit.summary())
    
    return coefs

coefs = generate_sig_model(doe_results)

                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     2367.
Date:                Thu, 10 Aug 2023   Prob (F-statistic):          2.44e-150
Time:                        12:03:52   Log-Likelihood:                -1743.3
No. Observations:                 146   AIC:                             3535.
Df Residuals:                     122   BIC:                             3606.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
1                  9.847e+04   2.03e+0

# Optimize
The function below optimizes the model using gurobi. Special care is taken to consider square terms and interactions so that they are constrained to equal the product of their first-order components.

In [8]:
#%% Create Gurobi model to optimize DOE results
import gurobipy as gp
from gurobipy import GRB

# Create Gurobi environment and suppress output
env = gp.Env(empty=True)
env.setParam("OutputFlag", 0)
env.start()

# Optimizes model based on the series of coefficients for each factor.
# Returns a Gurobi Model object and prints the optimal factor values
def optimize_model(coefs):
    model = gp.Model(env=env)
    model.setParam('NonConvex', 2) # To allow for quadratic equality constraints
    
    factors_dict = {}
    
    # Create gurobi variables
    for factor in coefs.index:
        if '^' not in factor and ' ' not in factor:
            factors_dict[factor] = model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=2, name=factor)
        else:
            factors_dict[factor] = model.addVar(vtype=GRB.CONTINUOUS, name=factor)
    
    # Add equality constraints
    for f in factors_dict:
        if f == '1': # Must maintain constant term, constrain this to 1
            factor = factors_dict[f]
            model.addConstr(factor == 1)
        if '^' in f: # Squared terms must be equal to the square of the first-order term
            first_order_f = re.search("^[a-z0-9_]*", str(f)).group(0) 
            first_order_factor = factors_dict[first_order_f]
            factor = factors_dict[f]
            model.addConstr(factor == first_order_factor**2)
        if ' ' in f: # Interaction terms must be the product of the first-order terms
            first_order_f1 = re.search("^[a-z0-9_]*", str(f)).group(0)
            first_order_f2 = re.search("[a-z0-9_]*$", str(f)).group(0)
            first_order_factor1 = factors_dict[first_order_f1]
            first_order_factor2 = factors_dict[first_order_f2]
            factor = factors_dict[f]
            model.addConstr(factor == first_order_factor1 * first_order_factor2)
            
    # Objective function
    model.setObjective(gp.quicksum(coefs[factor]*factors_dict[factor] for factor in factors_dict),
                        GRB.MAXIMIZE)    
    
    model.optimize()
    
    for var in model.getVars():
        print(var.varName, '=', var.x)
    print('objective value: ', model.objVal)
    
    return model

model = optimize_model(coefs)

1 = 1.0
wt_level = 2.0
sp_level = 0.0
b_level = 0.0
c1_level = 2.0
c2_level = 2.0
c3_level = 2.0
wt_level^2 = 4.0
wt_level sp_level = 0.0
wt_level b_level = 0.0
wt_level c1_level = 4.0
wt_level c2_level = 4.0
wt_level c3_level = 4.0
sp_level^2 = 0.0
sp_level b_level = 0.0
sp_level c1_level = 0.0
sp_level c2_level = 0.0
sp_level c3_level = 0.0
b_level^2 = 0.0
b_level c1_level = 0.0
b_level c2_level = 0.0
b_level c3_level = 0.0
c1_level c2_level = 4.0
c2_level^2 = 4.0
c2_level c3_level = 4.0
objective value:  2019496.3903095562


As the scaled values range from 0 to 2, it can be seen in the results that the optimal model is at a corner point, so the DOE is re-run below with new parameter values. The wind turbine number is set to 1, as the results of the first DOE indicate that 1 wind turbine is better than 0. Because they're so expensive, it is assumed that a 2nd wind turbine isn't an option. Because of this, the size of the DOE can be reduced to eliminate wt = 0. The new DOE design is saved in DOE2.xlsx. The other parameters are adjusted to be centered around their previous optimal values.

In [9]:
parameters2 = {
    "wt_list" : [1], # number of 1MW wind turbines
    "sp_list" : [2000, 5000, 8000], # area in m2 of solar panels
    "b_list" : [263, 516, 1144], # battery sizes in kW
    "c1_list" : [1, 2, 3], # constants for battery setpoint eqn
    "c2_list" : [1, 2, 3],
    "c3_list" : [0, 1, 2]
    }

# New DOE which removes the wind turbine feature and assumes it is fixed at 1
doe2 = pd.read_excel("DOE2.xlsx")

# Run DOE
doe_results2, _ = doef.run_doe(doe2, parameters2, show_run_status = False)

coefs2 = generate_sig_model(doe_results2, remove_var='wt_level')

model2 = optimize_model(coefs2)

                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     2903.
Date:                Thu, 10 Aug 2023   Prob (F-statistic):           4.64e-77
Time:                        12:12:47   Log-Likelihood:                -719.29
No. Observations:                  73   AIC:                             1469.
Df Residuals:                      58   BIC:                             1503.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
1                  1.754e+06   2937.09

Re-center the parameters again and re-run.

In [10]:
parameters3 = {
    "wt_list" : [1], # Only the case with 1 wind turbine is considered
    "sp_list" : [4000, 6500, 9000], # area in m2 of solar panels
    "b_list" : [0, 263, 516], # battery sizes in kW
    "c1_list" : [2, 3, 4], # constants for battery setpoint eqn
    "c2_list" : [2, 3, 4],
    "c3_list" : [-1, 0, 1]
    }

# DOE form doesn't change, so no new DOE is loaded

# Run DOE
doe_results3, _ = doef.run_doe(doe2, parameters3, show_run_status = False)

coefs3 = generate_sig_model(doe_results3, remove_var='wt_level')

model3 = optimize_model(coefs3)

                            OLS Regression Results                            
Dep. Variable:                 profit   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     830.9
Date:                Thu, 10 Aug 2023   Prob (F-statistic):           4.63e-62
Time:                        12:21:43   Log-Likelihood:                -696.40
No. Observations:                  73   AIC:                             1415.
Df Residuals:                      62   BIC:                             1440.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
1                  1.953e+06   1773.42

The results suggest it is optimal to have no battery. This could be reasonable, as it suggests the cost of the battery is too high to be offset by the energy from grid we must buy when renewable energy production is low. As all the constants relate to parameters for determining battery set point, they are no longer relevant in the model and will be set to 0 for the next calculation. Below, we calculate the expected results at this optimal level. Note that the optimal profit below deviates slightly from the value calculated above. This is due to the error between the actual and predicted values.

In [11]:
from weather_energy_components import years

parameters_final = {
    "wt_list" : [1], # Only the case with 1 wind turbine is considered
    "sp_list" : [6500], # area in m2 of solar panels
    "b_list" : [0], # battery sizes in kW
    "c1_list" : [0], # constants for battery setpoint eqn
    "c2_list" : [0],
    "c3_list" : [0]
    }

run = pd.Series([0, 0, 0, 0, 0, 0], ["wt_level", "sp_level", "b_level", "c1_level",
                                     "c2_level", "c3_level"])

profit, revenue, opex, capex, total_sx, e_to_grid, e_from_grid \
    = doef.run_scenario(forecast_store, parameters_final, run)
    
print("Profit (€/yr): ", round(profit))
print("Revenue (€/yr): ", round(revenue))
print("Opex (€/yr): ", round(opex))
print("Capex (€/yr): ", round(capex))
print("Sulfur (kmol/yr): ", round(total_sx/years/1000))
print("Energy sold to grid (MW/yr): ", round(e_to_grid/years/1000, 1))
print("Energy purchased from grid (MW/yr): ", round(e_from_grid/years/1000, 1))

Profit (€/yr):  2014687
Revenue (€/yr):  2304598
Opex (€/yr):  21730
Capex (€/yr):  268182
Sulfur (kmol/yr):  261
Energy sold to grid (MW/yr):  6.0
Energy purchased from grid (MW/yr):  86.9
